In [16]:
%matplotlib inline
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, scale
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('data/train.csv')

In [3]:
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,total_acc,initial_list_status,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,record_id
0,15000.0,36 months,11.99,498.15,B,B3,Quality Assurance Specialist,4 years,MORTGAGE,70000.0,...,32.0,f,0.0,1.0,INDIVIDUAL,0.0,0.0,295215.0,20500.0,453246940
1,3725.0,36 months,6.03,113.38,A,A1,NaN,NaN,MORTGAGE,52260.0,...,9.0,f,0.0,1.0,INDIVIDUAL,0.0,0.0,25130.0,14200.0,453313687
2,16000.0,36 months,11.14,524.89,B,B2,KIPP NYC,3 years,RENT,67500.0,...,22.0,f,0.0,1.0,INDIVIDUAL,0.0,193.0,41737.0,19448.0,453283543
3,4200.0,36 months,13.33,142.19,C,C3,Receptionist,< 1 year,MORTGAGE,21600.0,...,19.0,w,0.0,1.0,INDIVIDUAL,0.0,165.0,28187.0,14500.0,453447199
4,6500.0,36 months,12.69,218.05,B,B5,Medtox Laboratories,10+ years,RENT,41000.0,...,12.0,f,0.0,1.0,INDIVIDUAL,0.0,NaN,NaN,NaN,453350283


In [12]:
data = pd.DataFrame()
data['annual_inc'] = df['annual_inc']
data['dti'] = df['dti']
data['int_rate'] = df['int_rate']
label_encoder = LabelEncoder()
data['sub_grade'] = label_encoder.fit_transform(df['sub_grade'])

data['tot_cur_bal'] = df['tot_cur_bal']
data['tot_cur_bal'].fillna(data['tot_cur_bal'].notnull().min(), inplace=True)
data['loan_status'] = df['loan_status']

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200189 entries, 0 to 200188
Data columns (total 5 columns):
annual_inc     200189 non-null float64
dti            200189 non-null float64
int_rate       200189 non-null float64
sub_grade      200189 non-null int32
tot_cur_bal    200189 non-null object
dtypes: float64(3), int32(1), object(1)
memory usage: 6.9+ MB


In [17]:
X = data.drop(['loan_status'], axis=1)
y = data['loan_status']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train = scale(X_train)
X_test = scale(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int32, float64, object were all converted to float64 by the scale function.
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int32, float64, object were all converted to float64 by the scale function.
  """


In [20]:
from sklearn.cluster import KMeans


kmeans = KMeans(n_clusters=3, random_state=32)
preds_train = kmeans.fit_predict(X_train)

In [26]:
train_0 = np.array([])
train_1 = np.array([])
train_2 = np.array([])

train_0_y = []
train_1_y = []
train_2_y = []

y_train = list(y_train)
i = 0

for cluster in preds_train:    
    if cluster == 0:
        train_0 = np.append(train_0, list(X_train[i]))
        train_0_y.append(y_train[i])
    elif cluster == 1:
        train_1 = np.append(train_1, list(X_train[i]))
        train_1_y.append(y_train[i])
    elif cluster == 2:
        train_2 = np.append(train_2, list(X_train[i]))
        train_2_y.append(y_train[i])
    i += 1    
train_0 = train_0.reshape(int(train_0.size / 5), 5)
train_1 = train_1.reshape(int(train_1.size / 5), 5)
train_2 = train_2.reshape(int(train_2.size / 5), 5)

In [27]:
TRAIN_train_0, TEST_train_0, TRAIN_train_0_y, TEST_train_0_y = train_test_split(train_0, train_0_y, test_size=0.3, random_state=42)
TRAIN_train_1, TEST_train_1, TRAIN_train_1_y, TEST_train_1_y = train_test_split(train_1, train_1_y, test_size=0.3, random_state=42)
TRAIN_train_2, TEST_train_2, TRAIN_train_2_y, TEST_train_2_y = train_test_split(train_2, train_2_y, test_size=0.3, random_state=42)


In [30]:
import xgboost as xgb


cls_0 = xgb.XGBClassifier()
cls_0.fit(TRAIN_train_0, TRAIN_train_0_y)
preds = cls_0.predict(TEST_train_0)
print('XGB: ')
print('Accuracy score: ' + str(metrics.accuracy_score(TEST_train_0_y, preds)))

print('Recall score: ' + str(metrics.recall_score(TEST_train_0_y, preds)))

print('Precision score: ' + str(metrics.precision_score(TEST_train_0_y, preds)))

print('Auc score: ' + str(metrics.roc_auc_score(TEST_train_0_y, preds)))

XGB: 
Accuracy score: 0.8532657861407635
Recall score: 0.9993641373463332
Precision score: 0.8537026978091616
Auc score: 0.5003001156447364


In [31]:
cls_1 = xgb.XGBClassifier()
cls_1.fit(TRAIN_train_1, TRAIN_train_1_y)
preds = cls_1.predict(TEST_train_1)
print('XGB: ')
print('Accuracy score: ' + str(metrics.accuracy_score(TEST_train_1_y, preds)))

print('Recall score: ' + str(metrics.recall_score(TEST_train_1_y, preds)))

print('Precision score: ' + str(metrics.precision_score(TEST_train_1_y, preds)))

print('Auc score: ' + str(metrics.roc_auc_score(TEST_train_1_y, preds)))

XGB: 
Accuracy score: 0.8450100219083578
Recall score: 1.0
Precision score: 0.8450100219083578
Auc score: 0.5


In [32]:
cls_2 = xgb.XGBClassifier()
cls_2.fit(TRAIN_train_2, TRAIN_train_2_y)
preds = cls_2.predict(TEST_train_2)
print('XGB: ')
print('Accuracy score: ' + str(metrics.accuracy_score(TEST_train_2_y, preds)))

print('Recall score: ' + str(metrics.recall_score(TEST_train_2_y, preds)))

print('Precision score: ' + str(metrics.precision_score(TEST_train_2_y, preds)))

print('Auc score: ' + str(metrics.roc_auc_score(TEST_train_2_y, preds)))

XGB: 
Accuracy score: 0.6543390213133258
Recall score: 0.9517121181053927
Precision score: 0.6622200028534742
Auc score: 0.52974714629596


In [33]:
preds_test = kmeans.predict(X_test)

In [34]:
test_0 = np.array([])
test_1 = np.array([])
test_2 = np.array([])

test_0_y = []
test_1_y = []
test_2_y = []

y_test = list(y_test)
i = 0
for cluster in preds_test:
    
    if cluster == 0:
        test_0 = np.append(test_0, list(X_test[i]))
        test_0_y.append(y_test[i])
    elif cluster == 1:
        test_1 = np.append(test_1, list(X_test[i]))
        test_1_y.append(y_test[i])
    elif cluster == 2:
        test_2 = np.append(test_2, list(X_test[i]))
        test_2_y.append(y_test[i])
    i += 1    
test_0 = test_0.reshape(int(test_0.size / 5), 5)
test_1 = test_1.reshape(int(test_1.size / 5), 5)
test_2 = test_2.reshape(int(test_2.size / 5), 5)


In [35]:
preds_1 = cls_0.predict(test_0)
preds_1 = list(preds_1)
print('XGB: ')
print('Accuracy score: ' + str(metrics.accuracy_score(test_0_y, preds_1)))

print('Recall score: ' + str(metrics.recall_score(test_0_y, preds_1)))

print('Precision score: ' + str(metrics.precision_score(test_0_y, preds_1)))

print('Auc score: ' + str(metrics.roc_auc_score(test_0_y, preds_1)))

XGB: 
Accuracy score: 0.8478069624230238
Recall score: 0.9991109794043562
Precision score: 0.8483895319577253
Auc score: 0.5007972115564827


In [36]:
preds_2 = cls_1.predict(test_1)
preds_2 = list(preds_2)
print('XGB: ')
print('Accuracy score: ' + str(metrics.accuracy_score(test_1_y, preds_2)))

print('Recall score: ' + str(metrics.recall_score(test_1_y, preds_2)))

print('Precision score: ' + str(metrics.precision_score(test_1_y, preds_2)))

print('Auc score: ' + str(metrics.roc_auc_score(test_1_y, preds_2)))

XGB: 
Accuracy score: 0.8498188819632543
Recall score: 1.0
Precision score: 0.8498188819632543
Auc score: 0.5


In [37]:
preds_3 = cls_2.predict(test_2)
preds_3 = list(preds_3)
print('XGB: ')
print('Accuracy score: ' + str(metrics.accuracy_score(test_2_y, preds_3)))

print('Recall score: ' + str(metrics.recall_score(test_2_y, preds_3)))

print('Precision score: ' + str(metrics.precision_score(test_2_y, preds_3)))

print('Auc score: ' + str(metrics.roc_auc_score(test_2_y, preds_3)))

XGB: 
Accuracy score: 0.6583865405229062
Recall score: 0.9491283223778222
Precision score: 0.6674706059692493
Auc score: 0.5310579287803868


In [38]:
preds = np.append(preds_1, preds_2)
preds = np.append(preds, preds_3)

test_y = np.append(test_0_y, test_1_y)
test_y = np.append(test_y, test_2_y)


print('Auc score: ' + str(metrics.roc_auc_score(test_y, preds)))

Auc score: 0.5242007754181375
